# backtestig algorytmicznej strategii zakupu CO2 na rok Y+2
## zakup w oparciu o średnią ruchomą i odchylenie standardowe

model symuluje systematyczny zakup EUA DEC-23 w oparciu o sygnały zakupowe wygenerowane po przekroczeniu aktalnej w danym dniu ceny średniej ruchomej lub odchylenia standardowego

In [93]:
import pandas as pd
import datetime
import plotly.express as px
import numpy as np
import plotly.graph_objects as go

#### Dane wejściowe:
Dla zadanego całościowego wolumenu do zakupienia w określonym przedziale czasowym obliczamy tygodniowy wolumen uprawnień. Jeżeli w danym tygodniu nie nastąpi sygnał do zakupu, wolumeny tygodniowe się akumulują i czekają aż taki sygnał nastąpi.

#### Jeżeli nastąpi sygnał z tytułu przekroczenia w dół średniej ruchomej, kupujemy 30% zakumulowanego wolumenu do zakupienia
#### Jeżeli nastąpi sygnał z tytułu przekroczenia w dół odchylenia standardowego, kupujemy 100% zakumulowanego wolumenu do zakupienia

In [94]:
vb=500000 #volumen do kupienia
d_start=datetime.date(2021,9,23) #pierwsza transakcja
d_end=datetime.date(2022,12,31) # ostatnia transackja
mean_start=datetime.date(2022,1,1) # od kiedy liczona jest średnia giełdowa
days = np.busday_count(d_start, d_end) #przybliżona liczba dni handlowych
weekly_vol=round(vb/days*5,-3) #zakup przypadający na jeden tydzień

#### funkcja obliczająca średnią ruchomą oraz odchylenie standardowe dla zadanej liczby historycznych cen wstecz (zmienna okienko) oraz współczynnika korygującego odchylenie standardowe (zmienna x)

In [95]:
df1=pd.read_excel('source data.xlsx', header=0, parse_dates=['date'])
df1=df1[df1['price']!=0]
df1=df1[df1['date']>=pd.Timestamp(d_start)]

def rollowanie(okienko,x):

    df=df1.copy()
    df['date'] = pd.to_datetime(df['date'])
    df['dzientyg']=df['date'].dt.dayofweek
    df.loc[df['dzientyg']==0, 'weekly volume'] = weekly_vol
    df['weekly volume'] = df['weekly volume'].fillna(0)
    df['odchylenie']=df['price'].rolling(okienko).std(ddof = 0)
    df['srednia']=df['price'].rolling(okienko).mean()
    df['upper']=df['srednia']+x*df['odchylenie']
    df['lower']=df['srednia']-x*df['odchylenie']
    
    return df

#### Optymalizacja parametrów
poniższy kod iteracyjnie oblicza wynikową cenę w portfelu EUA w zależności od zadanego okna historycznego do wyznaczenia średniej i odchylenia standardowego oraz współczynnika pomniejszającego/powiększającego odchylenie standardowe

In [96]:
licznosc=[]
mnoznik=[]
wolumen=[]
portfel=[]

vol_accum=0
value_accum=0
df['cena w portfelu']=0

for j in range (10,31):
    for k in np.arange(0.5,2.1,0.1):
        
        results=pd.DataFrame(columns=['liczność próby','mnożnik odchylenia','wolumen zakupiony','cena w portfelu'])
        
        df=rollowanie(j,k)

        df['due_vol']=df['weekly volume'].cumsum()
        df.loc[df['price'] < df['srednia'], 'buy signal'] = 1
        df.loc[df['price'] < df['lower'], 'buy signal'] = 2
        n_row=df.shape[0]

        vol_accum=0
        value_accum=0
        df['cena w portfelu']=0

        for i in range(0,n_row-1):

            global t_vol
            global t_value

            buy_signal=df.iloc[i,9]
            due_vol=df.iloc[i,8]
            price=df.iloc[i,1]

            if buy_signal==1:

                global t_vol
                global t_value
                t_vol=round(due_vol*0.3,-3)
                t_value=t_vol*price
                df.iloc[i,8]=due_vol-t_vol
                df.iloc[i+1,8]=due_vol-t_vol+df.iloc[i+1,3]

            elif buy_signal==2:
                t_vol=round(due_vol*1,-3)
                t_value=t_vol*price
                df.iloc[i,8]=due_vol-t_vol 
                df.iloc[i+1,8]=due_vol-t_vol+df.iloc[i+1,3]

            else:
                t_vol=0
                t_value=0
                df.iloc[i+1,8]=df.iloc[i,8]+df.iloc[i+1,3]

            vol_accum=vol_accum+t_vol
            value_accum=value_accum+t_value

            try:
                df.iloc[i,10]=value_accum/vol_accum
                df.iloc[i+1,10]=value_accum/vol_accum
            except:
                pass
            
        licznosc.append(j)
        mnoznik.append(k)
        wolumen.append(vol_accum)
        portfel.append(value_accum/vol_accum)
           
        #print(j,k,value_accum/vol_accum,vol_accum)
        #print(df[df['date']>=pd.Timestamp(mean_start)]['price'].mean())
        


#### Wykres obrazujący wyniki optymalizacji

In [92]:
fig2 = go.Figure(data=go.Scatter(
    y = mnoznik,
    x=licznosc,
    mode='markers',
    marker=dict(
        size=16,
        color=portfel, #set color equal to a variable
        colorscale='Viridis', # one of plotly colorscales
        showscale=True)))

fig2.update_layout(title='wynikowa cena EUA w portfelu w zależności od mnożnika i liczności próby ', 
                  title_x=0.5,
                    xaxis_title='liczność próby',
                    yaxis_title='współczynnik przez który mnożymy odchylenie standardowe')

fig2.show()

#### najniższe wartości ceny w portfelu otrzymujemy dla liczby historycznych notowań do wyznaczenia sygnałów równej 19, dla współczynnika modyfikującego odchylenie standardowe w przedziale 0.8 - 1. Kod poniżej wyznacza jeszcze raz przebieg zakupów dla optymalnych parametrów:

In [97]:
vol_accum=0
value_accum=0
df['cena w portfelu']=0
     
df=rollowanie(19,0.9)

df['due_vol']=df['weekly volume'].cumsum()
df.loc[df['price'] < df['srednia'], 'buy signal'] = 1
df.loc[df['price'] < df['lower'], 'buy signal'] = 2
n_row=df.shape[0]

vol_accum=0
value_accum=0
df['cena w portfelu']=0

for i in range(0,n_row-1):

    global t_vol
    global t_value

    buy_signal=df.iloc[i,9]
    due_vol=df.iloc[i,8]
    price=df.iloc[i,1]

    if buy_signal==1:

        global t_vol
        global t_value
        t_vol=round(due_vol*0.3,-3)
        t_value=t_vol*price
        df.iloc[i,8]=due_vol-t_vol
        df.iloc[i+1,8]=due_vol-t_vol+df.iloc[i+1,3]

    elif buy_signal==2:
        t_vol=round(due_vol*1,-3)
        t_value=t_vol*price
        df.iloc[i,8]=due_vol-t_vol 
        df.iloc[i+1,8]=due_vol-t_vol+df.iloc[i+1,3]

    else:
        t_vol=0
        t_value=0
        df.iloc[i+1,8]=df.iloc[i,8]+df.iloc[i+1,3]

    vol_accum=vol_accum+t_vol
    value_accum=value_accum+t_value

    try:
        df.iloc[i,10]=value_accum/vol_accum
        df.iloc[i+1,10]=value_accum/vol_accum
    except:
        pass

#### wyniki symulacji zakupów w czasie przedstawia poniższy wykres

In [98]:
fig = go.Figure(data=go.Scatter(x=df['date'], 
                                    y=df['price'],
                                    name='cena settlement'))

fig.add_trace(go.Scatter(x = df['date'],
                         y = df['srednia'],
                         line_color = 'black',
                         name = 'średnia ruchoma'))

fig.add_trace(go.Scatter(x = df['date'],
                         y = df['upper'],
                         line_color = 'black',
                         name = 'średnia + odchylenie',
                         line=dict(width=1),
                         opacity=0.1))

fig.add_trace(go.Scatter(x = df['date'],
                         y = df['lower'],
                         line_color = 'black',
                         name = 'srednia - odchylenie',
                         line=dict(width=1),
                        fill='tonexty',
                        fillcolor='rgba(255, 0, 0, 0.1)',
                        opacity=0.1))

fig.add_trace(go.Scatter(x = df['date'],
                         y = df['cena w portfelu'],
                         mode='markers',
                         line_color = 'green',
                         name = 'cena w portfelu',
                         line=dict(width=2),
                         opacity=1))

fig.update_layout(title='historia notowań DEC-23 od początku 2021 r.', 
                  title_x=0.5,
                    xaxis_title='data',
                    yaxis_title='EUA price [EUA/t]')

fig.update_yaxes(range = [50,100])

fig.show()